In [1]:
import pandas as pd
import json

with open("10_yr_timeline.json", "r") as file:
    data = json.load(file)

df = pd.json_normalize(data)
df.columns = ["date", "count"]
df["date"] = pd.to_datetime(df["date"])
df["count"] = pd.to_numeric(df["count"], errors="coerce")
# Normalize to percentage of maximum (0-100 scale)
df["count"] = (df["count"] / df["count"].max()) * 100

In [2]:
cop_events = [
    {
        "name": "COP21",
        "location": "Paris, France",
        "start": "2015-11-30",
        "end": "2015-12-12",
    },
    {
        "name": "COP22",
        "location": "Marrakech, Morocco",
        "start": "2016-11-07",
        "end": "2016-11-18",
    },
    {
        "name": "COP23",
        "location": "Bonn, Germany",
        "start": "2017-11-06",
        "end": "2017-11-17",
    },
    {
        "name": "COP24",
        "location": "Katowice, Poland",
        "start": "2018-12-03",
        "end": "2018-12-14",
    },
    {
        "name": "COP25",
        "location": "Madrid, Spain",
        "start": "2019-12-02",
        "end": "2019-12-13",
    },
    {
        "name": "COP26",
        "location": "Glasgow, UK",
        "start": "2021-10-31",
        "end": "2021-11-12",
    },
    {
        "name": "COP27",
        "location": "Sharm El-Sheikh, Egypt",
        "start": "2022-11-06",
        "end": "2022-11-20",
    },
    {
        "name": "COP28",
        "location": "Dubai, UAE",
        "start": "2023-11-30",
        "end": "2023-12-12",
    },
    {
        "name": "COP29",
        "location": "Baku, Azerbaijan",
        "start": "2024-11-11",
        "end": "2024-11-22",
    },
    {
        "name": "COP30",
        "location": "Belém, Brazil",
        "start": "2025-11-10",
        "end": "2025-11-21",
    },
]

In [3]:
import plotly.graph_objects as go

# Catppuccin Mocha color palette

mocha = {
    "crust": "#11111b",
    "surface0": "#313244",
    "text": "#cdd6f4",
    "subtext0": "#a6adc8",
    "blue": "#89b4fa",
    "lavender": "#b4befe",
    "mauve": "#cba6f7",
}

fig = go.Figure()

rolled = df.set_index("date").rolling(30)["count"].mean()

# Annual average bars
annual_avg = df.groupby(df["date"].dt.year)["count"].mean().reset_index()
annual_avg.columns = ["year", "count"]

# Create proper date ranges for each year (centered on July 1st for better visualization)
bar_dates = [pd.Timestamp(f"{year}-07-01") for year in annual_avg["year"]]

fig.add_trace(
    go.Bar(
        x=bar_dates,
        y=annual_avg["count"],
        marker_color=mocha["mauve"],
        opacity=0.3,
        name="Annual Average",
        width=365 * 24 * 60 * 60 * 950,  # Width in milliseconds for full year
    )
)

# Add the rolling average line
fig.add_trace(
    go.Scatter(
        x=rolled.index,
        y=rolled.values,
        mode="lines",
        name="30-day average",
        line=dict(color=mocha["blue"], width=1),
    )
)

# Add vertical lines and labels for COP events
for event in cop_events:
    end_date = pd.Timestamp(event["end"])
    location = event["location"].split(",")[0]  # Get city name only

    fig.add_vline(
        x=end_date,
        line=dict(color=mocha["subtext0"], dash="dash", width=1),
        opacity=0.5,
    )
    fig.add_annotation(
        x=end_date,
        y=100,
        text=f"{event['name']}<br>{location}",
        textangle=-90,
        showarrow=False,
        align="right",
        font=dict(size=9, color=mocha["subtext0"]),
        xanchor="right",
        yanchor="top",
    )

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(gridcolor=mocha["surface0"]),
    # width=636,
    # height=400,
    # Legend at bottom
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color=mocha["subtext0"]),
    ),
)

# Set ylim 0-100
fig.update_yaxes(range=[0, 100])

# Label all years on y
fig.update_xaxes(
    dtick="M12",
    tickformat="%Y",
)

# Hide grid, set smaller fonts
fig.update_xaxes(showgrid=False, tickfont=dict(size=10))
fig.update_yaxes(
    showgrid=False,
    title=dict(text="Normalized article count", font=dict(size=10), standoff=5),
    tickfont=dict(size=10),
    # ticksuffix="%",
)

title = "Global climate coverage<br>"
sup_title = "<sup>Articles in NewsAPI.ai top 50 sources (2015-2025)</sup>"

fig.update_layout(
    title=dict(text=title + sup_title),
)

# Trim border
fig.update_layout(margin=dict(l=60, r=30, t=100, b=30))

# Round averages on hover
fig.update_traces(
    hovertemplate="%{y:.1f}%",
)

# increase gap between xticks and plot
fig.update_xaxes(
    ticklabelstandoff=10,
)
fig.show()

# Save to HTML with download button only
fig.write_html(
    "climate_ts.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)

In [4]:
import plotly.graph_objects as go

# Catppuccin Mocha color palette
mocha = {
    "crust": "#11111b",
    "surface0": "#313244",
    "text": "#cdd6f4",
    "subtext0": "#a6adc8",
    "blue": "#89b4fa",
    "lavender": "#b4befe",
    "mauve": "#cba6f7",
}

fig = go.Figure()

rolled = df.set_index("date").rolling(30)["count"].mean()

# Annual average bars
annual_avg = df.groupby(df["date"].dt.year)["count"].mean().reset_index()
annual_avg.columns = ["year", "count"]

# Create proper date ranges for each year (centered on July 1st for better visualization)
bar_dates = [pd.Timestamp(f"{year}-07-01") for year in annual_avg["year"]]

fig.add_trace(
    go.Bar(
        x=bar_dates,
        y=annual_avg["count"],
        marker_color=mocha["mauve"],
        opacity=0.3,
        name="Annual Average",
        width=365 * 24 * 60 * 60 * 950,  # Width in milliseconds for full year
    )
)

# Add the rolling average line
fig.add_trace(
    go.Scatter(
        x=rolled.index,
        y=rolled.values,
        mode="lines",
        name="90-day average",
        line=dict(color=mocha["blue"], width=1),
    )
)

# Add vertical lines and labels for COP events
for event in cop_events:
    end_date = pd.Timestamp(event["end"])
    location = event["location"].split(",")[0]  # Get city name only

    fig.add_vline(
        x=end_date,
        line=dict(color=mocha["subtext0"], dash="dash", width=1),
        opacity=0.5,
    )
    fig.add_annotation(
        x=end_date,
        y=100,
        text=f"{location}",
        textangle=-90,
        showarrow=False,
        align="right",
        font=dict(size=9, color=mocha["subtext0"]),
        xanchor="right",
        yanchor="top",
    )

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(gridcolor=mocha["surface0"]),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color=mocha["subtext0"]),
    ),
)

# Set ylim 0-100
fig.update_yaxes(range=[0, 100])


# Hide grid, set smaller fonts
fig.update_xaxes(showgrid=False, tickfont=dict(size=10))
fig.update_yaxes(
    showgrid=False,
    title=dict(text="Normalized article count", font=dict(size=10), standoff=5),
    tickfont=dict(size=10),
)

title = "Global climate coverage<br>"
sup_title = "<sup>Articles in NewsAPI.ai top 50 sources (2015-2025)</sup>"

fig.update_layout(
    title=dict(text=title + sup_title),
)

# Trim border
fig.update_layout(margin=dict(l=30, r=10, t=70, b=10))

# Round averages on hover
fig.update_traces(
    hovertemplate="%{y:.1f}%",
)

fig.show()

# Save to HTML with download button only
fig.write_html(
    "climate_ts_mobile.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)

In [5]:
df["pre_glasgow"] = df["date"] < pd.Timestamp("2021-10-31")

# pct Change pre-post Glasgow
print("Pre-Glasgow average articles per day:", df[df["pre_glasgow"]]["count"].mean())
print("Post-Glasgow average articles per day:", df[~df["pre_glasgow"]]["count"].mean())
print(
    "Pct change pre-post Glasgow:",
    (df[~df["pre_glasgow"]]["count"].mean() - df[df["pre_glasgow"]]["count"].mean())
    / df[df["pre_glasgow"]]["count"].mean()
    * 100,
)

Pre-Glasgow average articles per day: 13.160409991958439
Post-Glasgow average articles per day: 29.515837783353707
Pct change pre-post Glasgow: 124.27749440472691


In [6]:
# Year on year change
yearly = df.set_index("date").resample("YE")["count"].mean()
yearly_pct_change = yearly.pct_change() * 100
yearly_pct_change = yearly_pct_change.dropna()


# Colours, highlight negative bars
negative_years = yearly_pct_change[yearly_pct_change < 0].index.year.tolist()
colors = [
    mocha["lavender"] if year not in negative_years else mocha["mauve"]
    for year in yearly_pct_change.index.year
]


# Plotly bar chart of yearly_pct_change
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=yearly_pct_change.index.year,
        y=yearly_pct_change.values,
        marker_color=colors,
        name="Year-on-year change (%)",
    )
)
fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(gridcolor=mocha["surface0"]),
    title="Average daily articles (Y-o-Y % change)<br><sup>NewsAPI.ai top 50 sources (2016-2025)</sup>",
    yaxis_title="Year-on-year change (%)",
)

# Show all years on x axis
fig.update_xaxes(
    tickmode="array",
    tickvals=yearly_pct_change.index.year,
)

# Sort margins
fig.update_layout(margin=dict(l=70, r=30, t=70, b=30))

# Set hover to 2 decimal places, in year
fig.update_traces(hovertemplate="%{x}: %{y:.2f}%")

# Axis standoff
fig.update_xaxes(ticklabelstandoff=0)

fig.update_yaxes(
    ticksuffix="%",
    tickfont=dict(size=10),
)
fig.show()

fig.write_html(
    "climate_yearly_pct_change.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)

In [7]:
cops = pd.DataFrame(cop_events)
cops["start"] = pd.to_datetime(cops["start"])
cops["end"] = pd.to_datetime(cops["end"])

cop_effect = []
for _, row in cops.iterrows():
    # Get average count during COP event
    mask = (df["date"] >= row["start"]) & (df["date"] <= row["end"])
    avg_count = df.loc[mask, "count"].mean()

    # Get average over preceding six months
    six_months_prior = row["start"] - pd.DateOffset(months=12)
    prior_mask = (df["date"] >= six_months_prior) & (df["date"] < row["start"])
    prior_avg_count = df.loc[prior_mask, "count"].mean()

    cop_effect.append(
        {
            "name": row["name"]
            + "<br>"
            + f"<sup>{row['location'].split(',')[0]}</sup>",
            "avg_during_cop": avg_count,
            "avg_prior_6_months": prior_avg_count,
            "difference": avg_count - prior_avg_count,
            "pct_change": ((avg_count - prior_avg_count) / prior_avg_count) * 100,
        }
    )
cop_effect_df = pd.DataFrame(cop_effect).set_index("name")
cop_effect_df

,avg_during_cop,avg_prior_6_months,difference,pct_change
name,,,,
COP21<br><sup>Paris</sup>,25.036747,8.703544,16.333202,187.661507
COP22<br><sup>Marrakech</sup>,22.505308,12.261146,10.244161,83.549784
COP23<br><sup>Bonn</sup>,19.278132,13.949045,5.329087,38.203957
COP24<br><sup>Katowice</sup>,19.713376,11.058895,8.654480,78.258091
COP25<br><sup>Madrid</sup>,27.717622,16.324928,11.392694,69.787101
COP26<br><sup>Glasgow</sup>,64.262616,17.161330,47.101287,274.461754
COP27<br><sup>Sharm El-Sheikh</sup>,48.365180,27.416456,20.948725,76.409310
COP28<br><sup>Dubai</sup>,50.240078,31.693569,18.546509,58.518208
COP29<br><sup>Baku</sup>,42.653928,33.142947,9.510981,28.696848


In [16]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=cop_effect_df.index,
        y=cop_effect_df["pct_change"],
        marker=dict(color=mocha["mauve"]),
        name="COP effect (%)",
    )
)

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(gridcolor=mocha["surface0"], title="COP vs. prior 12 months (%)"),
    showlegend=False,
    title="The COP Effect: COP coverage increase<br><sup>Articles in NewsAPI.ai top 50 sources (2015-2025)</sup>",
    margin=dict(l=70, r=30, t=70, b=30),
)

# Set ylim
fig.update_yaxes(range=[0, 300])

# Set hover as 0 decimal places (rounded to nearest whole percent)
fig.update_traces(hovertemplate=f"%{{y:.0f}}%")

# Lower font size of x axis labels
fig.update_xaxes(showgrid=True, tickfont=dict(size=10))
fig.update_yaxes(showgrid=True)

fig.show()

fig.write_html(
    "cop_effect.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)

In [9]:
cop_effect_df["pct_change"].median()

np.float64(73.09820559860157)

In [10]:
# Pre-Glasgow average per day vs. 2021, 2022, 2023, 2024 and 2025
pre_glasgow_avg = df[df["date"] < pd.Timestamp("2021-10-31")]["count"].mean()
post_glasgow_avgs = {}
for year in range(2021, 2026):
    start_date = pd.Timestamp(f"{year}-01-01")
    end_date = pd.Timestamp(f"{year}-12-31")
    if year == 2021:
        start_date = pd.Timestamp("2021-10-31")  # Start from COP26 date
    yearly_avg = df[(df["date"] >= start_date) & (df["date"] <= end_date)][
        "count"
    ].mean()
    post_glasgow_avgs[year] = yearly_avg
    pct_change = ((yearly_avg - pre_glasgow_avg) / pre_glasgow_avg) * 100

In [17]:
# Plot plotly bar chart of post_glasgow_avg vs subsequent years (absolute values)
fig = go.Figure()

# Do pre-Glasgow average as a separate bar
fig.add_trace(
    go.Bar(
        x=["Pre-Glasgow Average"],
        y=[pre_glasgow_avg],
        marker_color=mocha["lavender"],
        name="Pre-Glasgow Average",
    )
)

fig.add_trace(
    go.Bar(
        x=list(post_glasgow_avgs.keys()),
        y=list(post_glasgow_avgs.values()),
        marker_color=mocha["mauve"],
        name="Average articles per day",
    )
)
fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(
        gridcolor=mocha["surface0"],
        title="Normalized article count",
    ),
    title="Average daily climate articles<br><sup>Articles in NewsAPI.ai top 50 sources (2015-2025)</sup>",
    # Set legend off
    showlegend=False,
    margin=dict(l=70, r=30, t=70, b=30),
)

# Round averages on hover
fig.update_traces(
    hovertemplate="%{y:.1f}%",
)

avg_2025 = 387
avg_pre_glasgow = 207
avg_2022 = 444
print(
    "Pct change pre-Glasgow to 2025:",
    (avg_2025 - avg_pre_glasgow) / avg_pre_glasgow * 100,
)
print("Pct change 2025 to 2022:", (avg_2022 - avg_2025) / avg_2025 * 100)

# Set ylim 0-100
fig.update_yaxes(range=[0, 50])

fig.show()
fig.write_html(
    "climate_pre_post_glasgow.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)

Pct change pre-Glasgow to 2025: 86.95652173913044
Pct change 2025 to 2022: 14.728682170542637


In [19]:
# Plot avg_during_cop vs avg_prior_6_months from cop_effect_df as grouped bar chart, 2021-2025
fig = go.Figure()
to_plot = cop_effect_df.iloc[5:]

fig.add_trace(
    go.Bar(
        x=to_plot.index,
        y=to_plot["avg_during_cop"],
        name="Average During COP",
        marker_color=mocha["mauve"],
    )
)
fig.add_trace(
    go.Bar(
        x=to_plot.index,
        y=to_plot["avg_prior_6_months"],
        name="Average Prior 6 Months",
        marker_color=mocha["lavender"],
    )
)

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(
        gridcolor=mocha["surface0"],
        title="Normalized article count",
    ),
    title="COP vs. prior 12 months<br><sup>Articles in NewsAPI.ai top 50 sources</sup>",
    margin=dict(l=60, r=20, t=70, b=20),
    barmode="group",
)

# Set hover to 1 decimal place with %
fig.update_traces(hovertemplate="%{y:.1f}%")

# Legend at bottom
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color=mocha["subtext0"]),
    )
)

# Set ylim
fig.update_yaxes(range=[0, 80])

fig.write_html(
    "cop_effect_grouped.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)
# Smaller labels on x and y axes
fig.update_xaxes(tickfont=dict(size=10))
fig.update_yaxes(tickfont=dict(size=10))
fig.show()

In [13]:
# Plot avg_during_cop vs avg_prior_6_months from cop_effect_df as grouped bar chart, 2021-2025
fig = go.Figure()
to_plot = cop_effect_df.iloc[5:]

fig.add_trace(
    go.Bar(
        x=to_plot.index,
        y=to_plot["avg_during_cop"],
        name="Average During COP",
        marker_color=mocha["mauve"],
    )
)
fig.add_trace(
    go.Bar(
        x=to_plot.index,
        y=to_plot["avg_prior_6_months"],
        name="Average Prior 6 Months",
        marker_color=mocha["lavender"],
    )
)

fig.update_layout(
    template="plotly_dark",
    paper_bgcolor=mocha["crust"],
    plot_bgcolor=mocha["crust"],
    font=dict(color=mocha["text"]),
    xaxis=dict(gridcolor=mocha["surface0"]),
    yaxis=dict(
        gridcolor=mocha["surface0"],
        title="Normalized article count",
    ),
    title="COP vs. prior 12 months (2021-2025)<br><sup>Articles in NewsAPI.ai top 50 sources</sup>",
    margin=dict(l=70, r=30, t=70, b=30),
    barmode="group",
)

# Set hover to 1 decimal place with %
fig.update_traces(hovertemplate="%{y:.1f}%")

# Legend at bottom
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color=mocha["subtext0"]),
    )
)

# Set ylim
fig.update_yaxes(range=[0, 100])

fig.write_html(
    "cop_effect_grouped.html",
    config={
        "displayModeBar": True,
        "displaylogo": False,
        "modeBarButtonsToRemove": [
            "pan2d",
            "lasso2d",
            "select2d",
            "zoom2d",
            "zoomIn2d",
            "zoomOut2d",
            "autoScale2d",
            "resetScale2d",
        ],
    },
)
# Smaller labels on x and y axes
fig.update_xaxes(tickfont=dict(size=10))
fig.update_yaxes(tickfont=dict(size=10))
fig.show()

In [14]:
import json

with open("10_yr_timeline_sources.json", "r") as f:
    sources_data = json.load(f)
sources_df = pd.json_normalize(sources_data)
sources_df
source_names = sorted(sources_df["Source title"].unique())
print(", ".join(source_names))

ABC ﻿TU DIARIO EN ESPAÑOL, ANSA.it, AP NEWS, BBC, CBC News, CBS News, CTV News, Channel NewsAsia, China Daily, Clarin, Deutsche Welle, EL MUNDO, EL PAÍS, El Universal Online, Estadão, Euronews English, Evening Standard, Fox News, France 24, Hindustan Times, Indian Express, La Jornada, La Repubblica.it, LaVanguardia, Le Figaro.fr, Le Monde.fr, Libération, Lietuvos Radijas ir Televizija, Los Angeles Times, Mail Online, National Post, Reuters, South China Morning Post, Star Tribune, The Boston Globe, The Daily Star, The Globe and Mail, The Guardian, The Hindu, The Independent, The New York Times, The Star , The Straits Times, The Telegraph, U.S. News and World Report, Washington Post, Yahoo, arab_news, infobae, uol.com.br
